Это улучшенная версия кода, которая гораздо быстрее и стабильнее обучается за счёт:

1. NumPy вместо вложенных списков (+скорость, -ошибки округления и переполнения)
Было:
w = [[random.uniform(-1, 1) for _ in range(layers[i])] for _ in range(layers[i + 1])]
Стало:
self.weights = [np.random.uniform(-1, 1, (layers[i], layers[i+1])) for i in range(len(layers)-1)]

2. Векторизованный forward и backkward (матричное умножение в NumPy в 10-100 раз быстрее циклов, обработка батчами, а не поэлементно)

3. Обработка батчей и нормализация градиентов

dw = self.a[i].T @ deltas[i] / m
db = np.mean(deltas[i], axis=0, keepdims=True)

Это предотвращает взрыв градиентов и делает обучение более плавным

4. Улучшенная ф-ция потерь
Было:
loss = sum([(outputs[i][j] - y_batch[i][j])**2 for j in range(len(outputs[i]))])
Стало:
loss = np.mean((output - y_batch)**2)

np.mean автоматически обрабатывает весь батч
устранены ошибки округления и nan, связанные с переполнением
стабильная и корректная работа MSE

5. Увеличена learning rate.

In [1]:
import pandas as pd

import random

import math

from sklearn.preprocessing import StandardScaler

import numpy as np

Необходимо реализовать класс Нейронная сеть, который позволяет инициализировать архитектуру полносвязной сети с возможностью передачи кол-ва нейронов, слоёв, функций активации, а также предполагает методы для обучения и валидации.

- инициализация архитектуры сети
- прямой проход
- обратный
- обучение модели
- предсказание
- валидация

In [2]:
class NeuralNetwork:
    def __init__(self, layers, activation_funcs, lr=0.01):
        self.layers = layers
        self.activation_funcs = activation_funcs
        self.lr = lr
        self.weights = [np.random.uniform(-1, 1, (layers[i], layers[i+1])) for i in range(len(layers)-1)]
        self.biases = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]

    def _activation(self, x, func):
        if func == 'relu':
            return np.maximum(0, x)
        elif func == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif func == 'tanh':
            return np.tanh(x)
        elif func == 'linear':
            return x

    def _activation_derivative(self, x, func):
        if func == 'relu':
            return (x > 0).astype(float)
        elif func == 'sigmoid':
            sig = self._activation(x, 'sigmoid')
            return sig * (1 - sig)
        elif func == 'tanh':
            return 1 - np.tanh(x)**2
        elif func == 'linear':
            return np.ones_like(x)

    def forward(self, X):
        self.z = []
        self.a = [X]

        for i in range(len(self.weights)):
            z = self.a[-1] @ self.weights[i] + self.biases[i]
            a = self._activation(z, self.activation_funcs[i])
            self.z.append(z)
            self.a.append(a)

        return self.a[-1]

    def backward(self, X, y):
        m = X.shape[0]
        output = self.a[-1]
        y = y.reshape(output.shape)
        deltas = [(output - y) * self._activation_derivative(self.z[-1], self.activation_funcs[-1])]

        for i in reversed(range(len(self.weights) - 1)):
            delta = deltas[0] @ self.weights[i+1].T * self._activation_derivative(self.z[i], self.activation_funcs[i])
            deltas.insert(0, delta)

        for i in range(len(self.weights)):
            dw = self.a[i].T @ deltas[i] / m
            db = np.mean(deltas[i], axis=0, keepdims=True)
            self.weights[i] -= self.lr * dw
            self.biases[i] -= self.lr * db

    def train(self, X_train, Y_train, epochs=100, batch_size=32):
        n_samples = X_train.shape[0]
        for epoch in range(epochs):
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            total_loss = 0

            for start in range(0, n_samples, batch_size):
                end = start + batch_size
                batch_idx = indices[start:end]
                X_batch = X_train[batch_idx]
                y_batch = Y_train[batch_idx]

                output = self.forward(X_batch)
                loss = np.mean((output - y_batch)**2)
                total_loss += loss * len(X_batch)

                self.backward(X_batch, y_batch)

            if epoch % 5 == 0:
                avg_loss = total_loss / n_samples
                print(f"Epoch {epoch}, MSE: {avg_loss:.4f}")

    def predict(self, X):
        return self.forward(X)

    def calculate_mae(self, X, y):
        pred = self.predict(X)
        return np.mean(np.abs(pred - y))

In [3]:
# Загрузка и подготовка данных
df = pd.read_csv("./additional/ParisHousing.csv")
X = df.drop('price', axis=1).values
Y = df['price'].values

In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Y_log = np.log1p(Y)

In [5]:
indices = list(range(len(X_scaled)))
random.shuffle(indices)
split = int(0.8 * len(X_scaled))
train_idx = indices[:split]
test_idx = indices[split:]

X_train = X_scaled[train_idx]
Y_train = Y_log[train_idx].reshape(-1, 1)
X_test = X_scaled[test_idx]
Y_test = Y_log[test_idx].reshape(-1, 1)

In [6]:
model = NeuralNetwork(
    layers=[X_train.shape[1], 64, 32, 1],
    activation_funcs=['relu', 'relu', 'linear'],
    lr=0.01
)
model.train(X_train, Y_train, epochs=150, batch_size=100)
mae = model.calculate_mae(X_test, Y_test)
print(f"MAE: {mae:.2f}")

Epoch 0, MSE: 76.4319
Epoch 5, MSE: 2.2469
Epoch 10, MSE: 1.5728
Epoch 15, MSE: 1.0662
Epoch 20, MSE: 0.6639
Epoch 25, MSE: 0.5878
Epoch 30, MSE: 0.5082
Epoch 35, MSE: 0.4555
Epoch 40, MSE: 0.3931
Epoch 45, MSE: 0.3536
Epoch 50, MSE: 0.5344
Epoch 55, MSE: 0.3212
Epoch 60, MSE: 0.3022
Epoch 65, MSE: 0.2416
Epoch 70, MSE: 0.2451
Epoch 75, MSE: 0.2188
Epoch 80, MSE: 0.1841
Epoch 85, MSE: 0.1863
Epoch 90, MSE: 0.1524
Epoch 95, MSE: 0.1569
Epoch 100, MSE: 0.1347
Epoch 105, MSE: 0.1254
Epoch 110, MSE: 0.1179
Epoch 115, MSE: 0.1203
Epoch 120, MSE: 0.0999
Epoch 125, MSE: 0.0932
Epoch 130, MSE: 0.0871
Epoch 135, MSE: 0.0831
Epoch 140, MSE: 0.0765
Epoch 145, MSE: 0.0706
MAE: 0.17
